### This Notebook will push a deployed model to OpenScale for monitoring its performance on payload data

### This will also configure Explainability, Fairness and Drift with appropriate thresholds.

### The following cell is a way to get the utility script required for this notebook. 
Since IBM CPD SaaS doesn't have a filesystem, this is the only reliable way to get scripts on the cloud environment. 
```
!rm -rf MLOps-CPD && git clone --quiet -b master https://github.com/IBM/MLOps-CPD.git
```

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
import os
from datetime import datetime

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_machine_learning import APIClient as wmlapiclient
from ibm_watson_studio_pipelines import WSPipelines
from ibm_aigov_facts_client import AIGovFactsClient
from botocore.client import Config
from ibm_watson_openscale.data_sets import DataSetTypes, TargetTypes
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
import ibm_boto3
import pandas as pd
import json

## DECLARE PIPELINE ENV VARIABLES

In [ ]:
CLOUD_API_KEY = os.getenv("cloud_api_key")
SPACE_ID = os.getenv("space_id")
MODEL_NAME = os.getenv("model_name")
DEPLOYMENT_NAME = os.getenv("deployment_name")

data_mart_id = os.getenv("data_mart_id")
service_provider_id = os.getenv("service_provider_id")

model_id = os.getenv("model_id")
deployment_id = os.getenv("deployment_id")
project_id = os.environ['PROJECT_ID']
training_file_name = os.environ['training_data_reference_file']

Below hidden cells were for debugging

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# The code was removed by Watson Studio for sharing.

## CREDENTIALS

### Load the validated training and test data from IBM COS 

```
## PROJECT COS 
AUTH_ENDPOINT = "https://iam.cloud.ibm.com/oidc/token"
ENDPOINT_URL = "https://s3.private.us.cloud-object-storage.appdomain.cloud"
API_KEY_COS = "xxx"
BUCKET_PROJECT_COS = "mlops-donotdelete-pr-qxxcecxi1dtw94"


##MLOPS COS
ENDPOINT_URL_MLOPS = "https://s3.jp-tok.cloud-object-storage.appdomain.cloud"
API_KEY_MLOPS = "xxx"
CRN_MLOPS = "xxx"
BUCKET_MLOPS  = "mlops-asset"
```

In [ ]:
# The code was removed by Watson Studio for sharing.

## Authenticate and Instantiate

In [ ]:
authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = APIClient(authenticator=authenticator,service_instance_id=data_mart_id)
wos_client.version

In [ ]:
def read_data_from_mlops_cos(key,json=False):
    def __iter__(self): return 0
    MLOPS_DATA_STORE_client = ibm_boto3.client(
        service_name='s3',
        ibm_api_key_id=API_KEY_MLOPS,
        ibm_service_instance_id=CRN_MLOPS,
        ibm_auth_endpoint=AUTH_ENDPOINT,
        config=Config(signature_version='oauth'),
        endpoint_url=ENDPOINT_URL_MLOPS)

    body = MLOPS_DATA_STORE_client.get_object(Bucket=BUCKET_MLOPS, Key=key)['Body']
    # add missing __iter__ method, so pandas accepts body as file-like object
    if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
    if json:
        gcf_df = body
    else:
        gcf_df = pd.read_csv(body)
    return gcf_df

In [ ]:
service_credentials = {
                  "apikey": CLOUD_API_KEY,
                  "url": "https://api.aiopenscale.cloud.ibm.com"
               }

## Add service provider

In [ ]:
WML_CREDENTIALS = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey": CLOUD_API_KEY
            }

In [ ]:
wos_client.service_providers.show()

## Determine if the deployment is in Prod Environment

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers

Check the tag for the service provider

In [ ]:
PROD_DEPLOYMENT = False
for service_provider in service_providers:
    deployment_space_id = service_provider.entity.deployment_space_id
    if deployment_space_id == SPACE_ID and service_provider.entity.operational_space_id == "production":
        PROD_DEPLOYMENT = True
print('The deployed model is in production environment:', PROD_DEPLOYMENT)

## Remove existing credit risk subscriptions

In [ ]:
wml_client = wmlapiclient(WML_CREDENTIALS)
wml_client.set.default_space(SPACE_ID)

In [ ]:
wml_client.repository.list()

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_id:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print("Deleted existing subscription for model", sub_model_id)

### Get Model Details

In [ ]:
# # DEPLOYMENT_NAME ='GCR_Model_DEPLOYMENT_NAME'

# #deployment_id = [model['metadata']['id'] for model in wml_client.deployments.get_details()['resources'] if model['metadata']['name']==DEPLOYMENT_NAME][-1]

deployment_id

deployment_url = [deployment["entity"]["status"]["online_url"]["url"] for deployment in wml_client.deployments.get_details()['resources'] if deployment['metadata']['name']==DEPLOYMENT_NAME][-1]
print(deployment_id,deployment_url)


published_model_details = wml_client.repository.get_details(model_id)
published_model_details

# wml_client.deployments.get_details()

## Prepare Eval Data

In [ ]:
test_data = read_data_from_mlops_cos('test_tfr.csv')
test_data = test_data.drop('Risk',axis=1)
features = test_data.columns.tolist()
categorical = test_data.select_dtypes('O').columns.tolist()
categorical, features

## Create the model subscription in OpenScale

In [ ]:
asset = Asset(
    asset_id=model_id,
    url=deployment_url,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    name=MODEL_NAME
)
asset_deployment = AssetDeploymentRequest(
    deployment_id=deployment_id,
    name=DEPLOYMENT_NAME,
    deployment_type=DeploymentTypes.ONLINE,
    url=deployment_url
)
training_data_reference = TrainingDataReference(
    type="cos",
    location=COSTrainingDataReferenceLocation(
        bucket=BUCKET_MLOPS,
        file_name=training_file_name
    ),
    connection=COSTrainingDataReferenceConnection.from_dict(
        {
            "resource_instance_id": CRN_MLOPS,
            "url": ENDPOINT_URL_MLOPS,
            "api_key": API_KEY_MLOPS,
            "iam_url": "https://iam.bluemix.net/oidc/token"
        }
    )
)

asset_properties_request = AssetPropertiesRequest(
    label_column="Risk",
    probability_fields=["probability"],
    prediction_field="prediction",
    feature_fields= features,
    categorical_fields= categorical,
    training_data_reference=training_data_reference,
    #training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
    )

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=asset,
        deployment=asset_deployment,
        asset_properties=asset_properties_request,background_mode = False).result
subscription_id = subscription_details.metadata.id
print(subscription_details)

## Verify the Subscription Registration

In [ ]:
wos_client.subscriptions.show()

## Get Payload Data ID

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

## Score the WML Endpoint

In [ ]:
payload_data = read_data_from_mlops_cos('test_tfr.csv')

In [ ]:
payload_data = payload_data.drop('Risk',axis=1)
fields = payload_data.columns.tolist()
values = [payload_data.values.tolist()[0]]

In [ ]:
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
json.dumps(payload_scoring)

In [ ]:
predictions = wml_client.deployments.score(deployment_id, payload_scoring)

print("Single record scoring result:", "\n fields:", predictions["predictions"][0]["fields"], "\n values: ", predictions["predictions"][0]["values"][0])

## Check if the Payload logging has taken place

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": predictions['predictions'][0]['fields'], "values":predictions['predictions'][0]['values']},
                   response_time=460
               )],background_mode = False)
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

## Enable Monitors for the Subscription

## Quality Monitor

In [ ]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 100
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .60
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

In [ ]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

## Get the Feedback Data for Monitoring

In [ ]:
from pprint import pprint
feedback_data = read_data_from_mlops_cos("hold_out_feedback_101.json",json=True)
feedback_data = json.loads(feedback_data.read())

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result

feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")
else:
    print("Feedback data set id:", feedback_dataset_id)

In [ ]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=feedback_data, background_mode=False)

In [ ]:
feedback_records_count = wos_client.data_sets.get_records_count(feedback_dataset_id)

In [ ]:
assert feedback_records_count >= 100, "Minimum feedback data size is set to 100, please add more feedback data!"

In [ ]:
time.sleep(5)

## Run Quality Monitor

In [ ]:
#pipeline metrics
openscale_metrics ={}

## Enable Fairness

In [ ]:
wos_client.monitor_instances.show()

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

## Enable Drift

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}


drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id

## Enable Explainability

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explain_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explain_monitor_details.metadata.id

### Payload Log the data 

In [ ]:
def pl_log(sub_id,request_data,response_data):
    # Retrieve the id of the payload logging data set
    SUBSCRIPTION_ID =sub_id
    payload_logging_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, target_target_id=SUBSCRIPTION_ID, target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id

    wos_client.data_sets.store_records(data_set_id=payload_logging_data_set_id, request_body=[PayloadRecord(request=request_data, response=response_data, response_time=460)])

    print("Payload Logging Successful")

In [ ]:
payload_test_data = read_data_from_mlops_cos('test_tfr.csv')
payload_test_data = payload_test_data.drop("Risk",axis=1)

fields = payload_test_data.columns.tolist()
values = payload_test_data.values.tolist()

In [ ]:
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
predictions = wml_client.deployments.score(deployment_id, payload_scoring)

In [ ]:
pl_log(subscription_id,payload_scoring,predictions)

In [ ]:
wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id)

## Enable MRM

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
}
mrm_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id='mrm',
    target=target,
    parameters=parameters
).result

mrm_instance_id = mrm_monitor_details.metadata.id

In [ ]:
import pandas as pd
test_data_mrm = read_data_from_mlops_cos('test_tfr.csv')
test_data_mrm.to_csv("gcr_mrm.csv", encoding="utf-8", index=False)

In [ ]:
import requests
def generate_access_token():
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = (
        f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={CLOUD_API_KEY}"
    )

    response = requests.post("https://iam.cloud.ibm.com/identity/token", headers=headers, data=data)

    return response.json()["access_token"]

In [ ]:
# evaluation_tests = ["fairness", "quality", "drift"]
# mrm_run_parameters = {"on_demand_trigger": True, "evaluation_tests": evaluation_tests, "publish_fact": "true"}
# wos_client.monitor_instances.run(monitor_instance_id=mrm_instance_id, triggered_by="user", background_mode=False, parameters=mrm_run_parameters)

# Non-prod evaluations

### Function to upload, evaluate and check the status of the evaluation

In [ ]:
def upload_and_evaluate(file_name, mrm_instance_id):
    
    print("Running upload and evaluate for {}".format(file_name))
    import json
    import time
    from datetime import datetime

    status = None
    monitoring_run_id = None
    GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES = 32
    GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL = 10
    
    if file_name is not None:
        
        headers = {}
        headers["Content-Type"] = "text/csv"
        headers["Authorization"] = "Bearer {}".format(generate_access_token())
        
        POST_EVALUATIONS_URL = service_credentials["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations?test_data_set_name={2}".format(data_mart_id, mrm_instance_id, file_name)

        with open(file_name) as file:
            f = file.read()
            b = bytearray(f, 'utf-8')
        

        response = requests.post(POST_EVALUATIONS_URL, data=bytes(b), headers=headers, verify=False)
    
        if response.ok is False:
            print("Upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))
            return
        
        headers = {}
        headers["Content-Type"] = "application/json"
        headers["Authorization"] = "Bearer {}".format(generate_access_token())

        GET_EVALUATIONS_URL = service_credentials["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations".format(data_mart_id, mrm_instance_id)
        
        for i in range(GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES):
        
            response = requests.get(GET_EVALUATIONS_URL, headers=headers, verify=False)
            if response.ok is False:
                print("Getting status of upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))
                return

            response = json.loads(response.text)
            if "metadata" in response and "id" in response["metadata"]:
                monitoring_run_id = response["metadata"]["id"]
            if "entity" in response and "status" in response["entity"]:
                status = response["entity"]["status"]["state"]
            
            if status is not None:
                print(datetime.utcnow().strftime('%H:%M:%S'), status.lower())
                if status.lower() in ["finished", "completed"]:
                    break
                elif "error" in status.lower():
                    print(response)
                    break

            time.sleep(GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL)

    return status, monitoring_run_id

### Run MRM

In [ ]:
if not PROD_DEPLOYMENT:
    upload_and_evaluate("gcr_mrm.csv", mrm_instance_id)

### Show the monitor metrics

In [ ]:
if not PROD_DEPLOYMENT:
    time.sleep(5)
    wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

In [ ]:
if not PROD_DEPLOYMENT:
    time.sleep(5)
    wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)


In [ ]:
if not PROD_DEPLOYMENT:
    time.sleep(5)
    wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

###  Run all the monitors

### Quality

In [ ]:
# run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

# time.sleep(5)
# wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

### Fairness

In [ ]:
# run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)
# time.sleep(5)
# wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

### Drift

In [ ]:
# drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)
# time.sleep(5)
# wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)


## Gather results for the last run 

In [ ]:
openscale_metrics ={}

In [ ]:
def get_monitor_metrics(config, deployment_name, monitor_type):
    wos_client = config["wos_client"]
    dict_monitor_instances = get_monitor_instances_by_deployment_name(config=config, deployment_name=deployment_name)
    start_time = datetime.now() - timedelta(days=7)
    end_time = datetime.now()

    monitor_instance_id = dict_monitor_instances[monitor_type]
    runs = wos_client.monitor_instances.runs.list(monitor_instance_id=monitor_instance_id).result.runs
    measurements = ibm_watson_openscale.base_classes.watson_open_scale_v2.Measurements(watson_open_scale=wos_client.monitor_instances._ai_client)
    for run in runs:
        run_id = run.to_dict()["metadata"]["id"]
        response = measurements.list(monitor_instance_id=monitor_instance_id, start=start_time, end=end_time, run_id=run_id)
        metrics = response.result.to_dict()["measurements"][0]["entity"]["values"][0]["metrics"]
        return metrics

In [ ]:
openscale_metrics

In [ ]:
# pipelines_client = WSPipelines.from_apikey(apikey="dhUSiBv8cezmf1NsfDX8ngsi1ruwf2DUE00bwpoHUlLk")
# pipelines_client.store_results(openscale_metrics)

In [ ]:
# wos_client.data_sets.show_records(data_set_id=feedback_dataset_id)

# Prod

## Fetch all monitor instances

In [ ]:
# Get the monitor instances IDs from OpenScale 
# Note: only run this if the monitor instances were already created before running this notebook

# headers = {}
# headers["Content-Type"] = "application/json"
# headers["Authorization"] = "Bearer {}".format(generate_access_token())

# MONITOR_INSTANCES_URL = "https://api.aiopenscale.cloud.ibm.com/openscale/{0}/v2/monitor_instances?target.target_id={1}&target.target_type=subscription".format(data_mart_id, subscription_id)
# print(MONITOR_INSTANCES_URL)

# response = requests.get(MONITOR_INSTANCES_URL, headers=headers)
# monitor_instances = response.json()["monitor_instances"]

# drift_monitor_instance_id = None
# quality_monitor_instance_id = None
# fairness_monitor_instance_id= None
# mrm_monitor_instance_id = None

# if monitor_instances is not None:
#     for monitor_instance in monitor_instances:
#         if "entity" in monitor_instance and "monitor_definition_id" in monitor_instance["entity"]:
#             monitor_name = monitor_instance["entity"]["monitor_definition_id"]
#             if "metadata" in monitor_instance and "id" in monitor_instance["metadata"]:
#                 id = monitor_instance["metadata"]["id"]
#                 if monitor_name == "drift":
#                     drift_monitor_instance_id = id
#                 elif monitor_name == "fairness":
#                     fairness_monitor_instance_id = id
#                 elif monitor_name == "quality":
#                     quality_monitor_instance_id = id
#                 elif monitor_name == "mrm":
#                     mrm_instance_id = id

In [ ]:
print("Quality monitor instance id - {0}".format(quality_monitor_instance_id))
print("Fairness monitor instance id - {0}".format(fairness_monitor_instance_id))
print("Drift monitor instance id - {0}".format(drift_monitor_instance_id))
print("MRM monitor instance id - {0}".format(mrm_instance_id))



## Function to get the monitoring run details



In [ ]:
def get_monitoring_run_details(monitor_instance_id, monitoring_run_id):
    
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Authorization"] = "Bearer {}".format(generate_access_token())
    
    MONITORING_RUNS_URL = "https://api.aiopenscale.cloud.ibm.com/openscale/{0}/v2/monitor_instances/{1}/runs/{2}".format(data_mart_id, monitor_instance_id, monitoring_run_id)
    response = requests.get(MONITORING_RUNS_URL, headers=headers, verify=False)
    return response.json()

## Run on-demand MRM

In [ ]:
if PROD_DEPLOYMENT:
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Authorization"] = "Bearer {}".format(generate_access_token())

    if mrm_instance_id is not None:
        MONITOR_RUN_URL ="https://api.aiopenscale.cloud.ibm.com/openscale/{0}/v2/monitor_instances/{1}/runs".format(data_mart_id, mrm_instance_id)
        payload = {
            "triggered_by": "user"
        }
        print("Triggering MRM computation with {}".format(MONITOR_RUN_URL))
        response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
        json_data = response.json()
        print()
        print(json_data)
        print()
        if "metadata" in json_data and "id" in json_data["metadata"]:
            mrm_monitoring_run_id = json_data["metadata"]["id"]
        print("Done triggering MRM computation")

In [ ]:
if PROD_DEPLOYMENT:
    mrm_run_status = None
    while mrm_run_status != 'finished':
        monitoring_run_details = get_monitoring_run_details(mrm_instance_id, mrm_monitoring_run_id)
        mrm_run_status = monitoring_run_details["entity"]["status"]["state"]
        if mrm_run_status == "error":
            print(monitoring_run_details)
            break
        if mrm_run_status != 'finished':
            print(datetime.utcnow().strftime('%H:%M:%S'), mrm_run_status)
            time.sleep(10)
    print(mrm_run_status)



## Run on-demand Quality

In [ ]:
if PROD_DEPLOYMENT:
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Authorization"] = "Bearer {}".format(generate_access_token())

    if quality_monitor_instance_id is not None:
        MONITOR_RUN_URL = "https://api.aiopenscale.cloud.ibm.com/openscale/{0}/v2/monitor_instances/{1}/runs".format(data_mart_id, quality_monitor_instance_id)
        payload = {
            "triggered_by": "user"
        }
        print("Triggering Quality computation with {}".format(MONITOR_RUN_URL))
        response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
        json_data = response.json()
        print()
        print(json_data)
        print()
        if "metadata" in json_data and "id" in json_data["metadata"]:
            quality_monitoring_run_id = json_data["metadata"]["id"]
        print("Done triggering Quality computation")

In [ ]:
if PROD_DEPLOYMENT:
    quality_run_status = None
    while quality_run_status != 'finished':
        monitoring_run_details = get_monitoring_run_details(quality_monitor_instance_id, quality_monitoring_run_id)
        quality_run_status = monitoring_run_details["entity"]["status"]["state"]
        if quality_run_status == "error":
            print(monitoring_run_details)
            break
        if quality_run_status != 'finished':
            print(datetime.utcnow().strftime('%H:%M:%S'), quality_run_status)
            time.sleep(10)
    print(quality_run_status)




## Run on-demand Fairness


In [ ]:
if PROD_DEPLOYMENT:
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Authorization"] = "Bearer {}".format(generate_access_token())

    if fairness_monitor_instance_id is not None:
        MONITOR_RUN_URL = "https://api.aiopenscale.cloud.ibm.com/openscale/{0}/v2/monitor_instances/{1}/runs".format(data_mart_id, fairness_monitor_instance_id)
        payload = {
            "triggered_by": "user"
        }
        print("Triggering fairness computation with {}".format(MONITOR_RUN_URL))
        response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
        json_data = response.json()
        print()
        print(json_data)
        print()
        if "metadata" in json_data and "id" in json_data["metadata"]:
            fairness_monitor_run_id = json_data["metadata"]["id"]
        print("Done triggering fairness computation")



In [ ]:
if PROD_DEPLOYMENT:
    fairness_run_status = None
    while fairness_run_status != 'finished':
        monitoring_run_details = get_monitoring_run_details(fairness_monitor_instance_id, fairness_monitor_run_id)
        fairness_run_status = monitoring_run_details["entity"]["status"]["state"]
        if fairness_run_status == "error":
            print(monitoring_run_details)
            break
        if fairness_run_status != 'finished':
            print(datetime.utcnow().strftime('%H:%M:%S'), fairness_run_status)
            time.sleep(10)
    print(fairness_run_status)